In [1]:
### Import & setup
import sys
sys.path.append('C:\\Users\\szb37\\My Drive\\Projects\\ADAPT\\ADAPT codebase\\')
import src.power as power
import src.config as config
import pandas as pd
from scipy import stats

# ignore guess_conf

In [2]:
### Ignore guess confidence
df_pop = power.Helpers.get_pop_data()
df_pop  = df_pop .drop(columns=['conf','gmg_se','gmg_ciL','gmg_ciH'])

gmg_C = df_pop.loc[(df_pop.trt=='C'), 'gmg'].to_numpy()
gmg_T = df_pop.loc[(df_pop.trt=='T'), 'gmg'].to_numpy()

ci_C = stats.ttest_1samp(gmg_C, popmean=0).confidence_interval(confidence_level=0.95)
ci_T = stats.ttest_1samp(gmg_T, popmean=0).confidence_interval(confidence_level=0.95)

df_ignore_gc = pd.DataFrame({
    'trt': ['C', 'T'],
    'gmg': [round(gmg_C.mean(),2), round(gmg_T.mean(),2)],
    'ciL': [round(ci_C.low,2), round(ci_T.low,2)],
    'ciH': [round(ci_C.high,2), round(ci_T.high,2)],
    'moe': [round((ci_C.high-ci_C.low)/2,2), round((ci_T.high-ci_T.low)/2,2)],
})
df_ignore_gc


,trt,gmg,ciL,ciH,moe
0,C,4.27,1.83,6.72,2.44
1,T,17.69,10.64,24.74,7.05


# guess_conf-to-CI

In [3]:
df_pop = power.Helpers.get_pop_data()
df_pop['conf']=3

df_pop['gmg_se'] = df_pop['conf'].map(config.conf_to_se)
df_pop['gmg_sd'] = df_pop['gmg_se'] # for sample=1, SE=SD
df_pop['gmg_ciL'] = df_pop['gmg']-1.96*df_pop['gmg_se'] 
df_pop['gmg_ciH'] = df_pop['gmg']+1.96*df_pop['gmg_se']
df_pop = df_pop.round({'gmg': 2, 'gmg_sd': 2, 'gmg_se': 2, 'gmg_ciL': 2, 'gmg_ciH': 2,})
df_pop = df_pop[['scenario', 'trial', 'pID', 'trt', 'gmg', 'conf', 'gmg_se', 'gmg_ciL', 'gmg_ciH']]
df_pop.head()

,scenario,trial,pID,trt,gmg,conf,gmg_se,gmg_ciL,gmg_ciH
0,POP,0,1008,C,5.0,3,7.14,-9.0,19.0
1,POP,0,1010,T,6.0,3,7.14,-8.0,20.0
2,POP,0,1011,C,0.0,3,7.14,-14.0,14.0
3,POP,0,1020,C,1.0,3,7.14,-13.0,15.0
4,POP,0,1021,T,0.0,3,7.14,-14.0,14.0


In [4]:
### Calculate POP CIs with guess_conf-to-CI approach
from itertools import product
import numpy as np
import scipy.stats as stats


df_gc2CI = get_df_combinedCI_fixed(df_pop, col_value='gmg', col_ciL='gmg_ciL', col_ciH='gmg_ciH',)
df_gc2CI = df_gc2CI.drop(columns=['scenario', 'trial'])
df_gc2CI

NameError: name 'get_df_combinedCI_fixed' is not defined

# guess_conf-to-weight

In [ ]:
### Calculate POP CIs with guess_conf-to-weigths approach
df_pop = power.Helpers.get_pop_data()
df_gc2w = power.Stats.get_df_weighted_gmgs(df_pop, col_value='gmg', col_conf='conf')
df_gc2w = df_gc2w.drop(columns=['scenario', 'trial'])
df_gc2w

,trt,gmg,ciL,ciH,moe
0,C,3.73,1.48,5.98,2.25
1,T,18.02,10.32,25.71,7.70
